In [63]:
%load_ext autoreload

import xarray as xr
import rasterio
import pandas as pd
import numpy as np
import osr
import arcpy
import datetime

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, cog, satfetcher, tools

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import gdvspectra

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# globals 
AWS_KEY = ''
AWS_SECRET = ''
STAC_ENDPOINT = 'https://explorer.sandbox.dea.ga.gov.au/stac/search'
RESULT_LIMIT = 250

In [ ]:
# take netcdf
# get latest dt
# get attributes for stac
# do cog

In [6]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\test2.nc")

In [75]:
# check
if 'time' not in ds:
    raise ValueError('No time dimension detected.')

# get bands
bands = list(ds.data_vars)
    
# check attributes
collections = list(ds.orig_collections)
bbox = list(ds.orig_bbox)
res = ds.res # use get xr res method
crs = ds.crs
in_slc_off = ds.orig_slc_off
resampling = ds.orig_resample
nodatavals = ds.nodatavals

# need to do
platform = 'Landsat'
dtype = 'int16'
fill_value = -999
snap = True
rescale = True
cell_align = 'Top-left'
chunk_size = -1



In [76]:
# get current clock time
dt_now = np.datetime64('now') # todo allow user override for specific dt

# get earliest, latest datetimes in dataset
dt_first = ds['time'].isel(time=0).values
dt_last = ds['time'].isel(time=-1).values

# conver to stac format
in_from_date = arc.datetime_to_string(pd.Timestamp(dt_last))
in_to_date = arc.datetime_to_string(pd.Timestamp(dt_now))


# check if xr dt less than now (will be for now, but not if override)
if dt_last < dt_now:
    
    # fetch cog
    feats = cog.fetch_stac_data(stac_endpoint=STAC_ENDPOINT, 
                                collections=collections, 
                                start_dt=in_from_date, 
                                end_dt=in_to_date, 
                                bbox=bbox,
                                slc_off=in_slc_off,
                                limit=RESULT_LIMIT)
    
    
else:
    print('No new scenes available. No sync required.')


#arc.datetime_to_string(dt_first)

Beginning STAC search for items. This can take awhile.
Searching collection: ga_ls5t_ard_3
Searching collection: ga_ls7e_ard_3
Searching collection: ga_ls8c_ard_3
Sorting result by time (old to new).
Found a total of 134 scenes.


In [7]:
in_nc = r"C:\Users\Lewis\Desktop\stac-9558e0f98ad918dbbcba681397cf91ee.nc"

wet_months = [1, 2, 3]
dry_months = [9, 10, 11]
in_fmask_flags = [1, 4, 5]
in_max_cloud = str(10)

ds = satfetcher.load_local_nc(nc_path=in_nc, 
                              use_dask=True, 
                              conform_nodata_to=np.nan)

Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: C:\Users\Lewis\Desktop\stac-9558e0f98ad918dbbcba681397cf91ee.nc
Netcdf converted to xarray dataset successfully.


In [8]:
ds = cog.remove_fmask_dates(ds=ds, 
                            valid_class=in_fmask_flags, 
                            max_invalid=in_max_cloud, 
                            mask_band='oa_fmask', 
                            nodata_value=np.nan, 
                            drop_fmask=True)  

Removing dates where too many invalid pixels.


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [53]:
arcpy.env.addOutputsToMap = False

# load netcdf and mdr
mdr = arcpy.Raster(r"C:\Users\Lewis\Desktop\stac-f7bbc2d21a1f66550458c7c09616e602.nc", True)

# check if multidim
is_multidim = mdr.isMultidimensional

# get mdim info
mdr_info = mdr.mdinfo

# get variable names
mdr_vars = mdr.variables

# get times
#mdr_times = mdr.getDimensionValues('x' 'time')

# export as mdr
#mdr.save("in_memory/out4.crf")
mdr.save(r"C:\Users\Lewis\Desktop\yesy\out5.crf")

#mdr_out = arcpy.md.MakeMultidimensionalRasterLayer(
    #in_multidimensional_raster = r"C:\Users\Lewis\Desktop\stac-f7bbc2d21a1f66550458c7c09616e602.nc",
    #out_multidimensional_raster_layer=r"C:\Users\Lewis\Desktop\yesy\stacer.crf",
    #variables=["nbart_red", "nbart_green", "nbart_blue"])

In [56]:
tif_layer = arcpy.CopyRaster_management(in_raster=r"C:\Users\Lewis\Desktop\stac-f7bbc2d21a1f66550458c7c09616e602.nc", 
                                        out_rasterdataset=r"out6.crf")

In [1]:
aprx = arcpy.mp.ArcGISProject('CURRENT')   
m = aprx.activeMap


#m.addDataFromPath(mdr)


#m.addDataFromPath("in_memory/out.crf")

In [90]:
lyr = m.listLayers('mdr.crf')[0]